In [1]:
pip install tqdm


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from tqdm import tqdm

tokens = {}
total_lines = sum(1 for _ in open("train16.txt", 'r', encoding='utf-8'))
with open("train16.txt", "r", encoding="utf-8") as fin:
    for line in tqdm(fin, total=total_lines, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue
        for token in line.split():
            if token not in tokens:
                tokens[token] = 0
            tokens[token] += 1

print("token dict:", json.dumps(tokens, indent=2))

Processing lines: 100%|██████████| 683536/683536 [00:03<00:00, 206296.32it/s]

token dict: {
  "add": 2196459,
  "mul": 2676318,
  "INT-": 1163047,
  "2": 877693,
  "Y": 469138,
  "x": 1893295,
  "Y'": 731671,
  "a8": 683792,
  "pow": 532528,
  "INT+": 1899959,
  "3": 416770,
  "1": 974956,
  "4": 373808,
  "ln": 254401,
  "exp": 294024,
  "sin": 216166,
  "cos": 227208,
  "sqrt": 164897,
  "tan": 134173,
  "5": 365705,
  "6": 132617,
  "div": 103891,
  "8": 85422,
  "acos": 38519,
  "sinh": 27826,
  "cosh": 30871,
  "asinh": 45049,
  "asin": 43009,
  "acosh": 43578,
  "atanh": 36033,
  "atan": 38988,
  "tanh": 11742,
  "0": 79757,
  "abs": 8754,
  "sign": 5527,
  "9": 52125,
  "7": 49635,
  "E": 26619,
  "pi": 8747,
  "cot": 141,
  "acot": 14,
  "coth": 1
}


In [4]:
# remove any lines with "cot", "acot", "coth" in them as they are low in frequency
# this will brings us to a neat 42 tokens including the SPECIAL tokens at tokenisation time!
total_lines = sum(1 for _ in open("train16.txt", 'r', encoding='utf-8'))
with open("train16.txt", "r", encoding="utf-8") as fin:
    with open("train16_clean.txt", "w", encoding="utf-8") as fout:
        for line in tqdm(fin, total=total_lines, desc="Processing lines"):
            line = line.strip()
            if not line:
                continue
            if "cot" in line or "acot" in line or "coth" in line:
                continue
            # rename "a8" to "c", "ln" to "log", "Y" to "y", "Y'" to "y'" cause those look good
            line = line.replace("a8", "c").replace("ln", "log").replace("Y", "y").replace("Y'", "y'")
            fout.write(line + "\n")

tokens = {}
total_lines = sum(1 for _ in open("train16_clean.txt", 'r', encoding='utf-8'))
with open("train16_clean.txt", "r", encoding="utf-8") as fin:
    for line in tqdm(fin, total=total_lines, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue
        for token in line.split():
            if token not in tokens:
                tokens[token] = 0
            tokens[token] += 1
print("token dict after cleaning:", json.dumps(tokens, indent=2))

Processing lines: 100%|██████████| 683446/683446 [00:03<00:00, 207478.23it/s]

token dict after cleaning: {
  "add": 2196187,
  "mul": 2676039,
  "INT-": 1162871,
  "2": 877565,
  "y": 469067,
  "x": 1893113,
  "y'": 731574,
  "c": 683702,
  "pow": 532449,
  "INT+": 1899762,
  "3": 416734,
  "1": 974800,
  "4": 373781,
  "log": 254389,
  "exp": 294014,
  "sin": 216147,
  "cos": 227196,
  "sqrt": 164894,
  "tan": 134167,
  "5": 365681,
  "6": 132617,
  "div": 103882,
  "8": 85421,
  "acos": 38518,
  "sinh": 27824,
  "cosh": 30866,
  "asinh": 45046,
  "asin": 43008,
  "acosh": 43576,
  "atanh": 36033,
  "atan": 38983,
  "tanh": 11742,
  "0": 79755,
  "abs": 8754,
  "sign": 5527,
  "9": 52125,
  "7": 49633,
  "E": 26619,
  "pi": 8741
}
